In [1]:
import re
from collections import defaultdict
from pathlib import Path

from tqdm.notebook import tqdm
from itertools import chain

import githelpers
import ghapicache
import release_notes

%load_ext autoreload
%autoreload 1
%aimport githelpers
%aimport ghapicache
%aimport release_notes

In [2]:
cached = ghapicache.GhApiCache(repo="g4vg")
ghapi = cached.api

In [3]:
# Load teams
TEAMS = {}
for t in tqdm(cached.teams()):
    TEAMS[t['name']] = release_notes.get_team(cached, t)

team_members = frozenset(chain(*(t.members for t in TEAMS.values())))
org_members = frozenset(m['login'] for m in cached.org_members())
TEAMS['affiliates'] = release_notes.Team(description="Associated with Celeritas but not core members",
                                         members=(org_members - team_members))

# Initialize the user cache
local_repo = Path("/Users/seth/Code/g4vg")
githelpers.REPO = local_repo
user_cache = release_notes.UserCache(cached, local_repo / "scripts/release/users.json")
for login in tqdm(org_members):
    user_cache[login]

# For G4VG don't include team roles
release_notes.TEAM_ROLES = {}

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

In [4]:
cached.flush()

In [5]:
all_tags = githelpers.git("tag", "--list")

# Filter tags matching the version pattern v(a.b.c) and extract the decimal component
#re_version = re.compile(r"^v(\d+\.\d+\.\d+)$")
re_version = re.compile(r"^v(\d+\.\d+\.\d+)$")
version_tuples = [tuple(int(v) for v in m.group(1).split('.')) for tag in all_tags
                   if (m := re_version.match(tag))]
#versions = [m.group(1) for tag in all_tags if (m := re_version.match(tag))]
# print(versions)

grouped_versions = defaultdict(list)
for major, minor, patch in version_tuples:
    grouped_versions[(major, minor)].append(patch)
grouped_versions.pop((0, 0), None)  # Remove the initial version (0.0.0)
# Find the last patch release for each minor release
last_version = {key: max(patches) for key, patches in grouped_versions.items()}

In [6]:
grouped_versions

defaultdict(list, {(0, 1): [0], (1, 0): [0, 1, 2, 3, 4]})

# Zenodo release

- All versions should list contributors since the initial last major release.
- Team members should be edited at the first release, then they'll be copied later.

In [7]:
print(release_notes.ReleaseMetadata.from_comprehensive_version(1,0,0))

ReleaseMetadata(release='1.0.0', merge_bases=['v1.0.0-dev^'], target_branch='v1.0.0')


In [ ]:
def load_contributions(release_md):
    prs = release_notes.PullRequestRange(release_md)
    count_contrib = release_notes.ContributionCounter(cached)
    for pr in tqdm(prs.pull_ids):
        count_contrib(pr)

    # Create author list
    return count_contrib.sorted()

make_zenodo_md = release_notes.ZenodoMetadataBuilder(user_cache=user_cache, teams=TEAMS, ghapi_cache=cached)
make_contrib = make_zenodo_md.make_zcontrib

In [9]:
import zenodoapi
%aimport zenodoapi

In [54]:
# Load the Zenodo token
#token_path = Path.home() / ".config/zenodo-sandbox-token"
token_path = Path.home() / ".config/zenodo-token"
with open(token_path) as f:
    zenodo_token = f.read().strip()

zenodo = zenodoapi.Zenodo(zenodo_token)
zenodo.api_url = "https://zenodo.org/api/"
#assert "sandbox" in zenodo.api_url

## Upload major versions

Major and minor versions should all look the same 

In [55]:
gh_release = release_notes.find_release(ghapi, "1.0.0")
tarball = release_notes.get_or_upload_tarball(cached, gh_release)

Loading https://github.com/celeritas-project/g4vg/releases/download/v1.0.0/g4vg-1.0.0.tar.gz from cached file data/ghapicache-downloads/346ac5ac958b39d4cd739d4fb512c79ca84deb05.gz
Found tarball in release assets


In [56]:
del grouped_versions[(0,1)]

KeyError: (0, 1)

In [57]:
urls = []
depositions = {}
for major, minor in tqdm(sorted(grouped_versions.keys())):
    release_md = release_notes.ReleaseMetadata.from_comprehensive_version(major, minor, 0)
    title = f"G4VG {major}.{minor}"
    deposition = zenodo.find_deposition(title)
    if deposition is None:
        gh_release = release_notes.find_release(ghapi, release_md.release)
        new_md = make_zenodo_md(load_contributions(release_md), release_md, gh_release)
        new_md["title"] = title

        # Create the Zenodo deposition
        deposition = zenodo.create_deposition(new_md)
    if not deposition.get_files():
        # Upload the release tarball
        tarball = release_notes.get_tarball(cached, gh_release)
        deposition.upload(tarball.content, tarball.name)
    depositions[(major, minor)] = deposition
    urls.append(deposition.html)

print("\n".join(urls))
print("""
IMPORTANT Checklist:
- Add community (CANNOT be done later)
- Update contributors based on release date
- Update licenses
""")

  0%|          | 0/1 [00:00<?, ?it/s]

https://zenodo.org/records/15450227

IMPORTANT Checklist:
- Add community (CANNOT be done later)
- Update contributors based on release date
- Update licenses



## Upload minor version

In [58]:
def create_minor_version(dep, *args):
    release_md = release_notes.ReleaseMetadata.from_comprehensive_version(*args)
    gh_release = release_notes.find_release(ghapi, release_md.release)

    # Get the Zenodo metadata    
    old_md = dep.data["metadata"]
    new_md = make_zenodo_md(load_contributions(release_md), release_md, gh_release)
    # Only update editors, not team members
    new_contrib = [u for u in new_md["contributors"] if u["type"] == "Editor"]
    old_contrib = [u for u in dep.md["contributors"] if u["type"] != "Editor"]
    new_md["contributors"] = new_contrib + old_contrib
    # Don't change the title
    new_md["title"] = dep.data["metadata"]["title"]
    # Add the body
    new_md["description"] = "\n\n".join(
        [
            old_md["description"],
            f"<h1>Version {release_md.release}</h1>",
            new_md["description"],
        ]
    )

    new_vers = dep.create_new_version()
    new_vers.update(new_md)
    # Upload the release
    tarball = release_notes.get_tarball(cached, gh_release)
    new_vers.upload(tarball.content, tarball.name)
    # The old tarball may still be there (this is buggy) so delete it
    new_vers.refresh()
    for file in new_vers.get_files():
        if file.filename != tarball.name:
            try:
                file.delete()
            except Exception as e:
                print(f"Failed to delete {file.filename}: {e}")
    new_vers.refresh()
    return new_vers

In [59]:
depositions

{(1, 0): ZenodoDeposition(id=15450227, title="G4VG 1.0")}

In [ ]:
# Create the next version update for each branch
for (major, minor), patches in grouped_versions.items():
    dep = depositions[(major, minor)].refresh()
    assert patches[0] == 0
    for patch in patches[1:]:
        ver_key = (major, minor, patch)
        try:
            # Access published version
            new_vers = depositions[ver_key]
        except KeyError:
            prevdep = zenodo.get_deposition(dep.get_latest_version().id)
            new_vers = create_minor_version(prevdep, *ver_key)
            #new_vers.md['creators'].append(make_contrib('mrguilima'))
            #new_vers.update()
            depositions[ver_key] = new_vers
            print(f"Created minor version {major}.{minor}.{patch}:", new_vers.html)
            break
        else:
            print(f"Existing minor version {major}.{minor}.{patch}:", new_vers.html)

Existing minor version 1.0.1: https://zenodo.org/deposit/15450415
Existing minor version 1.0.2: https://zenodo.org/deposit/15450420
Existing minor version 1.0.3: https://zenodo.org/deposit/15450421


  0%|          | 0/27 [00:00<?, ?it/s]

Created new version 15450422: https://zenodo.org/deposit/15450422
Updated deposition at https://zenodo.org/deposit/15450422 : G4VG 1.0
Uploaded g4vg-1.0.4.tar.gz: version ad7e1ded-1a4b-4138-8db4-a94e4af51957
Failed to delete g4vg-1.0.3.tar.gz: Expecting value: line 1 column 1 (char 0)
Updated deposition at https://zenodo.org/deposit/15450422 : G4VG 1.0
Created minor version 1.0.4: https://zenodo.org/deposit/15450422


In [63]:
cached.flush()

Saved cache to: data/ghapicache-celeritas-project-g4vg.json
